# ДЗ1. CLAP. Обучение проекции из аудио в текстовое пространство CLIP

**Описание задания**

В этом задании вы построите упрощённый вариант модели CLAP (Contrastive Language-Audio Pretraining):

- аудио прогоняется через предобученный аудио-энкодер (например, `LanguageBindAudio`, `CNN14/16` или другой);
- текстовое описание пропускается через предобученный текстовый энкодер CLIP;
- поверх аудио-векторов обучается линейный адаптер, который отображает аудио в то же пространство, что и текстовые эмбеддинги CLIP;
- обучение идёт по *контрастивному лоссу*, все энкодеры заморожены, обучаются только параметры аудио-проекции (и, при желании, температура в лоссе);
- качество полученного аудио-текстового пространства оценивается на задаче классификации / retrieval аудио по текстам на `AudioCaps`.

Идея оценки: если всё сделано правильно, для аудио и его описания косинусное сходство эмбеддингов будет выше, чем для аудио и нерелевантных текстов.


**Формулировка задач**

0. Выбор аудио-энкодера.
   Выберите и обоснуйте предобученный аудио-энбеддер:  
   - `LanguageBindAudio`,  
   - или CNN-модель (например, PANNs CNN14/16),  
   - или другой открытый аудио-энкодер, который выдаёт фиксированный эмбеддинг.

1. Подсчёт эмбеддингов.
   - Посчитайте аудио-векторы для всех аудио из `AudioCaps` с помощью выбранного энкодера.  
   - Посчитайте текстовые векторы для подписей с помощью `CLIP text encoder`.

2. Линейная аудио-проекция.
   - Реализуйте модель `AudioProjection`, переводящую аудио-эмбеддинг в размерность текстового эмбеддинга CLIP.

3. Контрастивное обучение.
   - Обучите аудио-проекцию на датасете `AudioCaps` по схеме аудио ↔ текст с контрастивным лоссом.  
   - Аудио-энкодер и CLIP должны быть полностью заморожены.

4. Оценка качества.
   - Оцените качество полученного аудио-текстового пространства на задаче классификации/ретривала аудио:  
     для каждого аудио найдите наиболее похожую текстовую подпись в батче/валидации и посчитайте `accuracy@1/3/10`.  
   - Сравните результаты с *случайным бейзлайном*.


### Сеттинг

> Подготовьте все необходимые импорты и загрузите необходимые данные.

In [1]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torchaudio
import soundfile as sf  # для загрузки .flac файлов
import pickle
from pathlib import Path
from tqdm import tqdm

from transformers import Wav2Vec2Processor, Wav2Vec2Model  # для wav2vec2
import clip  # для CLIP


In [2]:
# !kaggle datasets download nickkar30/audiocaps

In [3]:
# Для загрузки AudioCaps можно воспользоваться этим кодом

# !gdown --id 1FAVKNWXp5afgoNmclDwnj8j_OFTBRmIb -O audiocaps.zip
# !gdown --id 1fWZ0DN6IbSdjM_N0zTKyDQbypP053Y5l -O audiocaps.zip
# !unzip -q audiocaps.zip -d audiocaps

DATA_ROOT = "./audiocaps/audiocaps"
print("Files in DATA_ROOT:", os.listdir(DATA_ROOT))

Files in DATA_ROOT: ['audiocaps_test_new.tsv', 'val_texts.json', 'test_texts.json', 'audiocaps_train.tsv', 'audio', 'audiocaps_test.tsv', 'audiocaps_val_new.tsv', 'audiocaps_val.tsv']


### Задание 1. Подготовка аудио- и текстовых энкодеров (2 балла)

В этом задании вам нужно:

1. Выбрать аудио-энкодер и инициализировать его.
2. Инициализировать текстовый энкодер CLIP. Вы свободны выбирать самостоятельно, какой имеено.
3. Заморозить параметры обоих энкодеров (мы не дообучаем их, а учим только линейный адаптер).

Вы можете:

* использовать `LanguageBindAudio` (потребует установки репозитория и зависимостей);
* или подставить свою аудио-модель (главное - чтобы на выходе был вектор фиксированной размерности).


In [4]:
# your code here
# ┌(ಠ_ಠ)┘
# Инициализация устройств
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 1. Инициализация аудио-энкодера wav2vec2
print("Loading wav2vec2 model...")
audio_model_name = "facebook/wav2vec2-base"  # можно использовать также "facebook/wav2vec2-large"
audio_processor = Wav2Vec2Processor.from_pretrained(audio_model_name)
audio_model = Wav2Vec2Model.from_pretrained(audio_model_name).to(device)

# Заморозка параметров аудио-энкодера
for param in audio_model.parameters():
    param.requires_grad = False
audio_model.eval()
print(f"Audio encoder loaded. Hidden size: {audio_model.config.hidden_size}")
print("Note: wav2vec2 outputs sequence embeddings, will use mean pooling for fixed-size embedding")

# 2. Инициализация текстового энкодера CLIP
print("Loading CLIP model...")
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

# Заморозка параметров CLIP
for param in clip_model.parameters():
    param.requires_grad = False
clip_model.eval()

# Проверка размерности текстовых эмбеддингов CLIP
test_text = clip.tokenize(["test"]).to(device)
with torch.no_grad():
    test_text_embedding = clip_model.encode_text(test_text)
clip_text_dim = test_text_embedding.shape[-1]
print(f"CLIP model loaded. Text embedding dimension: {clip_text_dim}")

# Проверка размерностей
print("\n" + "="*50)
print("Models initialized successfully!")
print(f"Audio encoder (wav2vec2) hidden size: {audio_model.config.hidden_size}")
print(f"  -> After mean pooling: {audio_model.config.hidden_size} (fixed-size embedding)")
print(f"CLIP text embedding dim: {clip_text_dim}")
print("="*50)

Using device: cuda
Loading wav2vec2 model...


/home/yc-user/hse-fcs-multimodal-course/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Audio encoder loaded. Hidden size: 768
Note: wav2vec2 outputs sequence embeddings, will use mean pooling for fixed-size embedding
Loading CLIP model...
CLIP model loaded. Text embedding dimension: 512

Models initialized successfully!
Audio encoder (wav2vec2) hidden size: 768
  -> After mean pooling: 768 (fixed-size embedding)
CLIP text embedding dim: 512


### Задание 2. Предподсчёт аудио- и текстовых эмбеддингов (3 балла)

> Важный момент, который пригодится вам и в других домашних.

Чтобы не тратить время на многократный прогон энкодеров при обучении, следует:

1. Предварительно посчитывать аудио-эмбеддинги для каждого `.flac` в train/val/test.
2. Записывать их в файл формата `pickle` (например), где ключ - имя файла, значение - numpy-вектор.
3. Аналогично посчитать текстовые эмбеддинги для подписей через CLIP и совместить их с аудио.

Рекомендуемая структура:

* функция `extract_audio_vectors_with_checkpointing(...)` - обходит файлы, считает эмбеддинги, периодически делает чекпоинты;
* функция `extract_text_embeddings(texts, clip_model, clip_processor)` - возвращает список текстовых эмбеддингов;
* функция `process_dataset(...)` - читает `.tsv`, мержит аудио-эмбеддинги и текстовые, сохраняет список словарей вида  
  `{"uniq_id": ..., "audio_embedding": ..., "text_embedding": ...}` в pickle.

> Вы вольны отходить от предлагаемой структуры.

In [5]:
# your code here (づ｡◕‿‿◕｡)づ
# Базовые функции для обработки аудио и текста

def load_audio_file(audio_path, target_sr=16000):
    """
    Загружает аудио файл и ресемплирует до целевой частоты дискретизации.
    
    Args:
        audio_path: путь к аудио файлу (.flac)
        target_sr: целевая частота дискретизации (по умолчанию 16kHz для wav2vec2)
    
    Returns:
        waveform: numpy array с аудио данными, shape (n_samples,)
    """
    try:
        # Загружаем аудио файл через soundfile (работает с .flac без torchcodec)
        waveform, sample_rate = sf.read(audio_path, dtype='float32')
        
        # Конвертируем в моно (если стерео)
        if len(waveform.shape) > 1:
            waveform = np.mean(waveform, axis=1)
        
        # Ресемплинг до целевой частоты, если нужно
        if sample_rate != target_sr:
            # Используем torchaudio для ресемплинга (более точный)
            waveform_tensor = torch.from_numpy(waveform).unsqueeze(0)
            resampler = torchaudio.transforms.Resample(sample_rate, target_sr)
            waveform_tensor = resampler(waveform_tensor)
            waveform = waveform_tensor.squeeze(0).numpy()
        
        return waveform
    except Exception as e:
        print(f"Error loading audio file {audio_path}: {e}")
        return None


def extract_audio_embedding(audio_path, audio_processor, audio_model, device, data_root=None):
    """
    Извлекает фиксированный эмбеддинг из аудио файла с помощью wav2vec2.
    
    Args:
        audio_path: путь к аудио файлу (может быть относительным или абсолютным)
                   В TSV файлах путь имеет вид "audiocaps/audio/val/0.flac"
        audio_processor: Wav2Vec2Processor
        audio_model: Wav2Vec2Model (должна быть в eval режиме и на device)
        device: устройство (cuda/cpu)
        data_root: корневая директория (например, "./audiocaps/audiocaps")
    
    Returns:
        embedding: numpy array размерности [hidden_size] (768 для wav2vec2-base)
                   или None в случае ошибки
    """
    try:
        # Формируем полный путь
        if data_root:
            # Если путь уже содержит "audiocaps/", убираем его
            if audio_path.startswith("audiocaps/"):
                audio_path = audio_path[len("audiocaps/"):]
            full_path = Path(data_root) / audio_path
        else:
            full_path = Path(audio_path)
        
        # Проверяем существование файла
        if not full_path.exists():
            print(f"Audio file not found: {full_path}")
            return None
        
        # Загружаем аудио (Path можно использовать как строка для soundfile)
        waveform = load_audio_file(str(full_path))
        if waveform is None:
            return None
        
        # Обрабатываем через процессор wav2vec2
        inputs = audio_processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
        input_values = inputs.input_values.to(device)
        
        # Извлекаем эмбеддинги
        with torch.no_grad():
            outputs = audio_model(input_values)
            # Mean pooling по временной оси для получения фиксированного эмбеддинга
            # outputs.last_hidden_state shape: [batch, seq_len, hidden_size]
            audio_embedding = outputs.last_hidden_state.mean(dim=1)  # [batch, hidden_size]
            audio_embedding = audio_embedding.squeeze(0).cpu().numpy()  # [hidden_size]
        
        return audio_embedding
    
    except Exception as e:
        print(f"Error extracting embedding from {audio_path}: {e}")
        return None


def extract_text_embeddings(texts, clip_model, device, batch_size=32):
    """
    Извлекает текстовые эмбеддинги для списка текстов с помощью CLIP.
    
    Args:
        texts: список строк с текстами
        clip_model: CLIP модель (должна быть в eval режиме и на device)
        device: устройство (cuda/cpu)
        batch_size: размер батча для обработки
    
    Returns:
        embeddings: список numpy массивов, каждый размерности [text_dim] (512 для CLIP ViT-B/32)
    """
    all_embeddings = []
    
    # Обрабатываем тексты батчами
    for i in tqdm(range(0, len(texts), batch_size), desc="Extracting text embeddings"):
        batch_texts = texts[i:i + batch_size]
        
        # Токенизация
        text_tokens = clip.tokenize(batch_texts).to(device)
        
        # Извлечение эмбеддингов
        with torch.no_grad():
            text_embeddings = clip_model.encode_text(text_tokens)
            text_embeddings = text_embeddings.cpu().numpy()
        
        all_embeddings.extend(text_embeddings)
    
    return all_embeddings


In [6]:
# Тестирование базовых функций на одном примере
print("Testing basic functions...")
# Используем путь как в TSV файле: "audiocaps/audio/val/0.flac"
test_audio_path_tsv = "audiocaps/audio/val/0.flac"
# Или прямой путь для теста load_audio_file
test_audio_path_direct = Path(DATA_ROOT) / "audio/val/0.flac"

if test_audio_path_direct.exists():
    print(f"\n1. Testing load_audio_file on {test_audio_path_direct}")
    test_waveform = load_audio_file(test_audio_path_direct)
    if test_waveform is not None:
        print(f"   Audio loaded: shape={test_waveform.shape}, dtype={test_waveform.dtype}")
    
    print(f"\n2. Testing extract_audio_embedding with TSV-style path: {test_audio_path_tsv}")
    test_audio_emb = extract_audio_embedding(test_audio_path_tsv, audio_processor, audio_model, device, DATA_ROOT)
    if test_audio_emb is not None:
        print(f"   Audio embedding extracted: shape={test_audio_emb.shape}, dtype={test_audio_emb.dtype}")
else:
    print(f"  [WARN] Test audio file not found: {test_audio_path_direct}")

print("\n3. Testing extract_text_embeddings")
test_texts = ["A woman talks nearby as water pours", "Multiple clanging and clanking sounds"]
test_text_embs = extract_text_embeddings(test_texts, clip_model, device, batch_size=2)
if test_text_embs:
    print(f"Text embeddings extracted: {len(test_text_embs)} embeddings, shape={test_text_embs[0].shape}")

print("\nBasic functions are ready!")


Testing basic functions...

1. Testing load_audio_file on audiocaps/audiocaps/audio/val/0.flac
   Audio loaded: shape=(160000,), dtype=float32

2. Testing extract_audio_embedding with TSV-style path: audiocaps/audio/val/0.flac
   Audio embedding extracted: shape=(768,), dtype=float32

3. Testing extract_text_embeddings


Extracting text embeddings: 100%|██████████| 1/1 [00:00<00:00, 96.67it/s]

Text embeddings extracted: 2 embeddings, shape=(512,)

Basic functions are ready!


In [7]:
# Дополнительные функции для обработки аудио и текста

def extract_audio_vectors_with_checkpointing(
    tsv_path, 
    audio_processor, 
    audio_model, 
    device, 
    data_root,
    checkpoint_dir="./checkpoints",
    checkpoint_interval=1000,
    split_name=""
):
    """
    Извлекает аудио-эмбеддинги для всех аудио из TSV файла с поддержкой чекпоинтинга.
    
    Args:
        tsv_path: путь к TSV файлу с данными
        audio_processor: Wav2Vec2Processor
        audio_model: Wav2Vec2Model
        device: устройство (cuda/cpu)
        data_root: корневая директория для аудио файлов
        checkpoint_dir: директория для сохранения чекпоинтов
        checkpoint_interval: интервал сохранения чекпоинтов (количество файлов)
        split_name: имя сплита (train/val/test) для имени файла чекпоинта
    
    Returns:
        audio_embeddings_dict: словарь {uniq_id: numpy_array}
    """
    # Создаём директорию для чекпоинтов
    Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
    
    # Имя файла чекпоинта
    checkpoint_file = str(Path(checkpoint_dir) / f"audio_embeddings_{split_name}.pkl")
    
    # Загружаем существующий чекпоинт, если есть
    audio_embeddings_dict = {}
    if Path(checkpoint_file).exists():
        print(f"Loading checkpoint from {checkpoint_file}...")
        with open(checkpoint_file, 'rb') as f:
            audio_embeddings_dict = pickle.load(f)
        print(f"Loaded {len(audio_embeddings_dict)} embeddings from checkpoint")
    
    # Читаем TSV файл
    print(f"Reading TSV file: {tsv_path}")
    df = pd.read_csv(tsv_path, sep='\t')
    print(f"Total rows in TSV: {len(df)}")
    
    # Обрабатываем каждый аудио файл
    processed_count = 0
    skipped_count = 0
    error_count = 0
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Extracting audio embeddings ({split_name})"):
        uniq_id = row['uniq_id']
        audio_path = row['audio']
        
        # Пропускаем, если уже обработано
        if uniq_id in audio_embeddings_dict:
            skipped_count += 1
            continue
        
        # Извлекаем эмбеддинг
        audio_embedding = extract_audio_embedding(
            audio_path, 
            audio_processor, 
            audio_model, 
            device, 
            data_root
        )
        
        if audio_embedding is not None:
            audio_embeddings_dict[uniq_id] = audio_embedding
            processed_count += 1
        else:
            error_count += 1
            print(f" [WARN] Failed to extract embedding for uniq_id={uniq_id}, audio={audio_path}")
        
        # Сохраняем чекпоинт периодически
        if processed_count > 0 and processed_count % checkpoint_interval == 0:
            print(f"Saving checkpoint after {processed_count} processed files...")
            with open(checkpoint_file, 'wb') as f:
                pickle.dump(audio_embeddings_dict, f)
    
    # Финальное сохранение
    print("Saving final checkpoint...")
    with open(checkpoint_file, 'wb') as f:
        pickle.dump(audio_embeddings_dict, f)
    
    print("\n[OK] Audio embeddings extraction completed!")
    print(f"   - Processed: {processed_count}")
    print(f"   - Skipped (already in checkpoint): {skipped_count}")
    print(f"   - Errors: {error_count}")
    print(f"   - Total embeddings: {len(audio_embeddings_dict)}")
    
    return audio_embeddings_dict


def process_dataset(
    tsv_path,
    audio_processor,
    audio_model,
    clip_model,
    device,
    data_root,
    checkpoint_dir="./checkpoints",
    split_name="",
    checkpoint_interval=1000
):
    """
    Обрабатывает датасет: извлекает аудио- и текстовые эмбеддинги и сохраняет в pickle.
    
    Args:
        tsv_path: путь к TSV файлу
        audio_processor: Wav2Vec2 Processor
        audio_model: Wav2Vec2 Model
        clip_model: CLIP модель
        device: устройство (cuda/cpu)
        data_root: корневая директория для аудио файлов
        checkpoint_dir: директория для чекпоинтов
        split_name: имя сплита (train/val/test)
        checkpoint_interval: интервал сохранения чекпоинтов
    
    Returns:
        dataset: список словарей вида {"uniq_id": ..., "audio_embedding": ..., "text_embedding": ..., ...}
    """
    print(f"\n{'='*60}")
    print(f"Processing dataset: {split_name}")
    print(f"{'='*60}\n")
    
    # 1. Извлекаем аудио-эмбеддинги
    print("Step 1: Extracting audio embeddings...")
    audio_embeddings_dict = extract_audio_vectors_with_checkpointing(
        tsv_path,
        audio_processor,
        audio_model,
        device,
        data_root,
        checkpoint_dir,
        checkpoint_interval,
        split_name
    )
    
    # 2. Читаем TSV для получения текстов
    print("\nStep 2: Reading TSV file for texts...")
    df = pd.read_csv(tsv_path, sep='\t')
    texts = df['text'].tolist()
    uniq_ids = df['uniq_id'].tolist()
    audio_paths = df['audio'].tolist()
    
    # 3. Извлекаем текстовые эмбеддинги
    print("\nStep 3: Extracting text embeddings...")
    text_embeddings = extract_text_embeddings(texts, clip_model, device, batch_size=32)
    
    # 4. Объединяем данные
    print("\nStep 4: Combining audio and text embeddings...")
    dataset = []
    missing_audio_count = 0
    
    for i, uniq_id in enumerate(uniq_ids):
        if uniq_id in audio_embeddings_dict:
            dataset.append({
                "uniq_id": uniq_id,
                "audio_embedding": audio_embeddings_dict[uniq_id],
                "text_embedding": text_embeddings[i],
                "text": texts[i],
                "audio_path": audio_paths[i]
            })
        else:
            missing_audio_count += 1
            print(f" [WARN] Missing audio embedding for uniq_id={uniq_id}")
    
    # 5. Сохраняем финальный датасет
    output_file = os.path.join(checkpoint_dir, f"embeddings_{split_name}.pkl")
    print(f"\nStep 5: Saving combined dataset to {output_file}...")
    with open(output_file, 'wb') as f:
        pickle.dump(dataset, f)
    
    print("\n[OK] Dataset processing completed!")
    print(f"   - Total samples: {len(dataset)}")
    print(f"   - Missing audio embeddings: {missing_audio_count}")
    print(f"   - Saved to: {output_file}")
    
    return dataset



In [8]:
# Обработка датасетов
checkpoint_dir = "./checkpoints"

val_tsv = str(Path(DATA_ROOT) / "audiocaps_val_new.tsv")
val_dataset = process_dataset(
    val_tsv,
    audio_processor,
    audio_model,
    clip_model,
    device,
    DATA_ROOT,
    checkpoint_dir,
    split_name="val",
    checkpoint_interval=100
)

test_tsv = str(Path(DATA_ROOT) / "audiocaps_test_new.tsv")
test_dataset = process_dataset(
    test_tsv,
    audio_processor,
    audio_model,
    clip_model,
    device,
    DATA_ROOT,
    checkpoint_dir,
    split_name="test",
    checkpoint_interval=100
)

train_tsv = str(Path(DATA_ROOT) / "audiocaps_train.tsv")
train_dataset = process_dataset(
    train_tsv,
    audio_processor,
    audio_model,
    clip_model,
    device,
    DATA_ROOT,
    checkpoint_dir,
    split_name="train",
    checkpoint_interval=1000
)



Processing dataset: val

Step 1: Extracting audio embeddings...
Loading checkpoint from checkpoints/audio_embeddings_val.pkl...
Loaded 495 embeddings from checkpoint
Reading TSV file: audiocaps/audiocaps/audiocaps_val_new.tsv
Total rows in TSV: 495


Extracting audio embeddings (val): 100%|██████████| 495/495 [00:00<00:00, 20552.99it/s]


Saving final checkpoint...

[OK] Audio embeddings extraction completed!
   - Processed: 0
   - Skipped (already in checkpoint): 495
   - Errors: 0
   - Total embeddings: 495

Step 2: Reading TSV file for texts...

Step 3: Extracting text embeddings...


Extracting text embeddings: 100%|██████████| 16/16 [00:00<00:00, 78.97it/s]



Step 4: Combining audio and text embeddings...

Step 5: Saving combined dataset to ./checkpoints/embeddings_val.pkl...

[OK] Dataset processing completed!
   - Total samples: 495
   - Missing audio embeddings: 0
   - Saved to: ./checkpoints/embeddings_val.pkl

Processing dataset: test

Step 1: Extracting audio embeddings...
Loading checkpoint from checkpoints/audio_embeddings_test.pkl...
Loaded 963 embeddings from checkpoint
Reading TSV file: audiocaps/audiocaps/audiocaps_test_new.tsv
Total rows in TSV: 963


...


Step 4: Combining audio and text embeddings...

Step 5: Saving combined dataset to ./checkpoints/embeddings_test.pkl...

[OK] Dataset processing completed!
   - Total samples: 963
   - Missing audio embeddings: 0
   - Saved to: ./checkpoints/embeddings_test.pkl

Processing dataset: train

Step 1: Extracting audio embeddings...
Loading checkpoint from checkpoints/audio_embeddings_train.pkl...
Loaded 49490 embeddings from checkpoint
Reading TSV file: audiocaps/audiocaps/audiocaps_train.tsv
Total rows in TSV: 49490


Extracting audio embeddings (train): 100%|██████████| 49490/49490 [00:02<00:00, 23356.99it/s]


Saving final checkpoint...

[OK] Audio embeddings extraction completed!
   - Processed: 0
   - Skipped (already in checkpoint): 49490
   - Errors: 0
   - Total embeddings: 49490

Step 2: Reading TSV file for texts...

Step 3: Extracting text embeddings...


Extracting text embeddings: 100%|██████████| 1547/1547 [00:16<00:00, 91.34it/s]



Step 4: Combining audio and text embeddings...

Step 5: Saving combined dataset to ./checkpoints/embeddings_train.pkl...

[OK] Dataset processing completed!
   - Total samples: 49490
   - Missing audio embeddings: 0
   - Saved to: ./checkpoints/embeddings_train.pkl


### Задание 3. Линейный аудио-адаптер и контрастивный лосс (3 балла)


Теперь, когда у нас есть пары *audio_embedding, text_embedding*, реализуем:

1. Класс `AudioTextDataset`, который читает pickle с комбинированными эмбеддингами.
2. Линейную модель `AudioProjection`, переводящую аудио-эмбеддинг в размерность текстового.
3. Контрастивный лосс для аудио↔текст:
   - нормализовать эмбеддинги по L2;
   - посчитать матрицу сходства;
   - задать таргеты как `targets = arange(batch_size)`;
   - вычислить `CrossEntropyLoss` как для строк audio→text и для строк text→audio, усреднить.

Обучаем **только** `AudioProjection` (и, по желанию, параметр temperature).


In [ ]:
# your code here
# (╯°□°）╯︵ ┻━┻

### Задание 4. Оценка качества на задаче классификации аудио (2 балла)


Теперь нужно понять, насколько хорошо аудио-векторы после проекции "попадают" в пространство текстовых эмбеддингов:

1. Посчитайте проекции аудио для всех примеров в валидации.
2. Для каждого аудио найдите `top-k` наиболее похожих текстов по косинусному сходству (или скалярному произведению после L2-нормализации).
3. Посчитайте `accuracy@1`, `accuracy@3`, `accuracy@10`, т.е. долю случаев, когда "правильный" текст попал в топ-k.
4. Сравните с неким *случайным бейзлайном*: для каждого аудио выберите `k` случайных текстов и посчитайте такую же метрику.

> Важно: в батче класс "правильного" текста для i-го аудио - это индекс i (как в контрастивном лоссе).

In [ ]:
# your code here
# (⌐■_■)

### Вывод

Оформите, пожалуйста, небольшой вывод. Например, можно воспрользоваться следующим планом:

   * какую аудио-модель вы выбрали и почему;
   * как вели себя потери на обучении;
   * какие значения метрик получились и насколько они превосходят случайный baseline;
   * любые наблюдения (например, зависимость от числа эпох, размера батча и т.д.);
   * милые пожелания ассистенту/лектору, который будет это проверять.

your text here (ಠ.ಠ)